In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
# InvoiceNo: 주문 번호
# StockCode: 상품 코드
# Description: 상품 설명
# Quantity: 상품 수량
# InvoiceDate: 주문 날짜
# UnitPrice: 상품 단가
# CustomerID: 고객 ID
# Country: 국가

In [ ]:
# online_retail.csv 파일 불러오기
df = pd.read_csv(f"data/online_retail.csv")
df = df.dropna()
df.shape

In [ ]:
df['StockCodeNo'] = df['StockCode'].astype("category").cat.codes
# 상품 코드와 고객 ID로 이루어진 새로운 데이터프레임 생성
new_df = df[['StockCodeNo', 'CustomerID', 'Quantity']]

# NaN 값을 가지는 행 제거
new_df = new_df.dropna()

# 고객 ID와 상품 코드를 정수형으로 변환
new_df['CustomerID'] = new_df['CustomerID'].astype(int)
new_df['StockCodeNo'] = new_df['StockCodeNo'].astype(int)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df[['StockCodeNo', 'CustomerID', 'Quantity']], test_size=0.2)

In [ ]:
train_df.head()

MF 클래스는 nn.Module을 상속받아서 정의되며, n_users와 n_items는 각각 사용자와 아이템의 수이며, n_factors는 잠재 요인(latent factor)의 수입니다. 모델의 학습을 통해 잠재 요인이 학습됩니다.

nn.Embedding은 입력 텐서에 대해 임베딩을 수행하는 클래스입니다. 여기서는 각 사용자와 아이템을 임베딩합니다. user_factors와 item_factors는 각각 사용자와 아이템의 잠재 요인을 나타내는 행렬입니다. 임베딩된 사용자와 아이템 벡터는 forward 함수에서 곱해져서 예측 평점을 계산합니다.

forward 함수에서는 각 사용자와 아이템의 임베딩 벡터를 가져와서 내적(dot product)을 계산합니다. 내적의 결과는 각 사용자와 아이템의 잠재 요인 값들을 조합한 예측 평점을 나타냅니다. 따라서 모델의 출력은 사용자-아이템 쌍에 대한 예측 평점의 벡터가 됩니다.

In [ ]:
import torch
from torch import nn

In [ ]:
df['StockCodeNo'] = df['StockCode'].astype("category").cat.codes

In [ ]:
new_df = df.groupby(["CustomerID", "StockCodeNo"])["Quantity"].sum().reset_index()
new_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋을 train set과 test set으로 나눔
train_df, val_df = train_test_split(new_df, test_size=0.2)

# 사용자 수와 상품 수 계산
n_users = new_df['CustomerID'].nunique()
n_items = new_df['StockCodeNo'].nunique()

# 파이토치에서 사용하기 위해 데이터셋을 텐서로 변환
train_data = torch.FloatTensor(train_df.values)
val_data = torch.FloatTensor(val_df.values)

train_data = torch.tensor(train_data, dtype=torch.long)
test_data = torch.tensor(test_data, dtype=torch.long)

train_data.shape, val_data.shape, test_data[0]


train_loader = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=256, shuffle=False)

In [ ]:
# 사용자 ID와 제품 코드를 고유한 값으로 매핑
user_to_idx = {user: i for i, user in enumerate(train_df['CustomerID'].unique())}
item_to_idx = {item: i for i, item in enumerate(train_df['StockCodeNo'].unique())}

In [ ]:
print(n_users, n_items)

In [ ]:
class MF(nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super(MF, self).__init__()
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.item_factors = nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        user_embedding = self.user_factors(user)
        item_embedding = self.item_factors(item)
        return (user_embedding * item_embedding).sum(1)

model = MF(n_users, n_items, n_factors=20)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)

In [ ]:
for epoch in range(3):
    for train_batch in train_loader.dataset[:3]:
        user, item, rating = train_batch
        optimizer.zero_grad()
        output = model(user.unsqueeze(0), item.unsqueeze(0))
        loss = criterion(output, rating)
        print(output, loss)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        val_loss = 0.0
        for val_batch in val_loader.dataset:
            user, item, rating = val_batch
            output = model(user, item)
            val_loss += criterion(output, rating).item() * len(user)
        val_loss /= len(val_df)
        print('epoch: {}, validation RMSE loss: {:.4f}'.format(epoch+1, val_loss**0.5))

In [ ]:
# 각 제품의 인덱스를 제품 코드로 매핑
idx_to_item = {i: item for item, i in item_to_idx.items()}

# 특정 사용자에게 추천할 상위 10개의 제품 출력
user_idx = 0
user_items = set(train_df[train_df['user'] == user_idx]['item'])
scores = model(torch.LongTensor([user_idx]*len(item_to_idx)), torch.LongTensor(list(item_to_idx.values()))).detach().numpy()
item_indices = list(range(len(item_to_idx)))
item_scores = list(zip(item_indices, scores))
item_scores = sorted(item_scores, key=lambda x: x[1], reverse=True)
recommended_items = []
for item_idx, score in item_scores:
    item_code = idx_to_item[item_idx]
    if item_code not in user_items:
        recommended_items.append(item_code)
    if len(recommended_items) >= 10:
        break
print('Recommended items for user {}:'.format(user_idx))
print(recommended_items)
